# Using jmzqc via Maven

In order to use jmzqc as a library, add the following statement to your maven pom.xml file. Please check https://ms-quality-hub.github.io/jmzqc/dependency-info.html for details on how to add the dependency for other build tools.


In [7]:
%%loadFromPOM
<!-- for mzQC functionality -->
<dependency>
    <groupId>org.lifs-tools</groupId>
    <artifactId>jmzqc</artifactId>
    <version>1.0.0</version>
</dependency>
<!-- for mzML access -->
<dependency>
    <groupId>io.github.msdk</groupId>
    <artifactId>msdk-io-mzml</artifactId>
    <version>0.0.27</version>
</dependency>
<!-- for table-based operations -->
<dependency>
    <groupId>tech.tablesaw</groupId>
    <artifactId>tablesaw-core</artifactId>
    <version>0.43.1</version>
</dependency>
<!-- for plotting -->
<dependency>
    <groupId>org.knowm.xchart</groupId>
    <artifactId>xchart</artifactId>
    <version>3.8.2</version>
</dependency>
<!-- for logging -->
    <!--
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-simple</artifactId>
    <version>1.7.26</version>
</dependency>-->

Add the following imports to your Java file to use the MzQC classes and the validation messages.


In [8]:
import java.nio.*;
import java.nio.file.*;
import java.net.*;
import java.io.*;

In [10]:
var outputDir = new File("MTBLS1375");
try {
    outputDir.mkdirs();
    var cmd = "wget -r -l 1 -A .mzML --reject 'index.html*' -P MTBLS1375 -nc -np -nd -nH --cut-dirs=6 -e robots=off https://ftp.ebi.ac.uk/pub/databases/metabolights/studies/public/MTBLS1375/";
    var pb = new ProcessBuilder(cmd.split(" "));
    System.out.println("Downloading files with: " + pb.command());
    pb.inheritIO();
    Process p = pb.start();
    p.waitFor();
    System.out.println("Finished downloading of files!");
} catch (IOException | InterruptedException ex) {
    System.err.println("Exception:" + ex.getLocalizedMessage());
}

Finished downloading of files!


In [11]:
import io.github.msdk.MSDKException;
import io.github.msdk.MSDKRuntimeException;
import io.github.msdk.datamodel.ChromatogramType;
import io.github.msdk.io.mzml.MzMLFileImportMethod;
import io.github.msdk.io.mzml.data.MzMLRawDataFile;
import java.util.stream.Collectors;
var List<MzMLRawDataFile> mzMLData = Collections.emptyList();
try {
    var mzMLFilePaths = Files.list(outputDir.toPath()).collect(Collectors.toList());
    mzMLData = mzMLFilePaths.stream().map(path -> {
        try {
            return new MzMLFileImportMethod(path).execute();
        } catch (MSDKException ex) {
            throw new MSDKRuntimeException(ex);
        }
    }
    ).collect(Collectors.toList());
} catch (IOException ex) {
    System.err.println("Exception:" + ex.getLocalizedMessage());
}
System.out.println("Loaded "+mzMLData.size()+" files!");

[INFO] XMLStreamReaderImpl: Data buffer increased to 16384
[INFO] XMLStreamReaderImpl: Data buffer increased to 32768
[INFO] XMLStreamReaderImpl: Data buffer increased to 65536
[INFO] XMLStreamReaderImpl: Data buffer increased to 131072
[INFO] XMLStreamReaderImpl: Data buffer increased to 262144
[INFO] XMLStreamReaderImpl: Data buffer increased to 16384
[INFO] XMLStreamReaderImpl: Data buffer increased to 32768
[INFO] XMLStreamReaderImpl: Data buffer increased to 65536
[INFO] XMLStreamReaderImpl: Data buffer increased to 131072
[INFO] XMLStreamReaderImpl: Data buffer increased to 262144
[INFO] XMLStreamReaderImpl: Data buffer increased to 16384
[INFO] XMLStreamReaderImpl: Data buffer increased to 32768
[INFO] XMLStreamReaderImpl: Data buffer increased to 65536
[INFO] XMLStreamReaderImpl: Data buffer increased to 131072
[INFO] XMLStreamReaderImpl: Data buffer increased to 262144
[INFO] XMLStreamReaderImpl: Data buffer increased to 16384
[INFO] XMLStreamReaderImpl: Data buffer increased 

In [12]:
import org.lifstools.jmzqc.*;
import com.networknt.schema.*;
import java.util.AbstractMap.SimpleEntry;
import com.google.common.collect.Range;
var mzMLFormatParameter = new CvParameter("MS:1000584", null, "mzML format", null);
Map<InputFile, List<QualityMetric>> mzMLFileStats = mzMLData.stream().map((t) -> {
    System.out.println("Processing file: " + t.getName());

    var precursorMzRange = t.getChromatograms().stream().filter(chrom -> chrom.getChromatogramType() == ChromatogramType.MRM_SRM).map(chrom -> {
        var mzr = Range.singleton(chrom.getIsolations().get(0).getPrecursorMz());
        return mzr;
    }).filter(range -> range != null && !range.isEmpty()).reduce((l, r) -> l.span(r)).orElse(Range.singleton(0.0));
    var precursorMzRangeMetric = new QualityMetric("MS:4000069", null, "m/z acquisition range", Arrays.asList(precursorMzRange.lowerEndpoint(), precursorMzRange.upperEndpoint()), null);
    var numberOfChromatogramsMetric = new QualityMetric("MS:4000071", null, "number of chromatograms", t.getChromatograms().stream().filter(chrom -> chrom.getChromatogramType() == ChromatogramType.MRM_SRM).count(), null);
    var rtRange = t.getChromatograms().stream().map(
            chrom -> {
                return chrom.getRtRange();
            }
    ).reduce(
            (lrt, rrt) -> lrt.span(rrt)
    ).get();
    var rtRangeMetric = new QualityMetric("MS:4000070", null, "retention time acquisition range", Arrays.asList(rtRange.lowerEndpoint(), rtRange.upperEndpoint()), null);
    return new SimpleEntry<InputFile, List<QualityMetric>>(
            new InputFile(mzMLFormatParameter, Collections.emptyList(), t.getOriginalFile().get().toURI(), t.getName()),
            Arrays.asList(
                    numberOfChromatogramsMetric,
                    precursorMzRangeMetric,
                    rtRangeMetric
            )
    );
}).collect(Collectors.toMap(Map.Entry::getKey, Map.Entry::getValue));

Processing file: PBLK_02.mzML
Processing file: 30_BQC_3_A-2.mzML
Processing file: 71_LTR_2_B-1.mzML
Processing file: 76_TQC_B-2.mzML
Processing file: 91_PLASMA_H041_B-1.mzML
Processing file: 92_PLASMA_H041_B-2.mzML
Processing file: 131_PLASMA_H059_B-1.mzML
Processing file: 5_NIST_A-1.mzML
Processing file: 15_PLASMA_H010_A-1.mzML
Processing file: 144_TQC_B-1.mzML
Processing file: 139_RQC-2-20.mzML
Processing file: 46_PLASMA_H051_A-2.mzML
Processing file: 59_PLASMA_H057_A-1.mzML
Processing file: RQC-1-10.mzML
Processing file: 132_PLASMA_H059_B-2.mzML
Processing file: 21_PLASMA_H018_A-1.mzML
Processing file: 136_BQC_7_B-2.mzML
Processing file: 129_PLASMA_H058_B-1.mzML
Processing file: 116_BQC_5_B-2.mzML
Processing file: 107_PLASMA_H048_B-1.mzML
Processing file: RQC-1-40.mzML
Processing file: PBLK_03.mzML
Processing file: 56_PLASMA_H056_A-2.mzML
Processing file: ISTD_BuMe_01.mzML
Processing file: 70_LTR_1_B-2.mzML
Processing file: 25_PLASMA_H042_A-1.mzML
Processing file: 118_PLASMA_H052_B-

In [13]:
import com.fasterxml.jackson.annotation.JsonInclude.Include;
import com.fasterxml.jackson.core.JsonFactoryBuilder;
import com.fasterxml.jackson.core.JsonParser;
import com.fasterxml.jackson.core.JsonProcessingException;
import com.fasterxml.jackson.core.json.JsonReadFeature;
import com.fasterxml.jackson.databind.DeserializationContext;
import com.fasterxml.jackson.databind.JsonDeserializer;
import com.fasterxml.jackson.databind.ObjectMapper;
import com.fasterxml.jackson.databind.ObjectWriter;
import com.fasterxml.jackson.databind.SerializationFeature;
import com.fasterxml.jackson.databind.module.SimpleModule;
/* helper function to create formatted / pretty printed JSON */
public static ObjectWriter prepareJsonWriter() {
    JsonFactoryBuilder jfb = new JsonFactoryBuilder().
            enable(JsonReadFeature.ALLOW_TRAILING_COMMA);
    ObjectMapper mapper = new ObjectMapper(jfb.build());
    mapper.findAndRegisterModules();
    mapper.configure(SerializationFeature.WRITE_DATES_AS_TIMESTAMPS, false);
    mapper.configure(SerializationFeature.INDENT_OUTPUT, true);
    mapper.configure(JsonParser.Feature.ALLOW_COMMENTS, true);
    mapper.setSerializationInclusion(Include.NON_EMPTY);

    SimpleModule module = new SimpleModule();
    module.addDeserializer(OffsetDateTime.class, new JsonDeserializer<OffsetDateTime>() {
        @Override
        public OffsetDateTime deserialize(JsonParser jsonParser, DeserializationContext deserializationContext) throws IOException, JsonProcessingException {
            String value = jsonParser.getText();
            return Converter.parseDateTimeString(value);
        }
    });
    mapper.registerModule(module);
    return mapper.writerFor(Coordinate.class);
}

In [15]:
import java.time.OffsetDateTime;
MzQC mzQC = null;
var file = "MTBLS1375/MTBLS1375";
try {
    var analysisSoftware = new AnalysisSoftware("MS:1000799", null, "custom unreleased software tool", "jmzqc", new URI("https://github.com/MS-Quality-hub/jmzqc"), "1.0.0-RC1");
    List<BaseQuality> bqs = mzMLFileStats.entrySet().stream().map((t) -> {
        Metadata metadata = new Metadata(Arrays.asList(analysisSoftware), Collections.emptyList(), Arrays.asList(t.getKey()), null);
        return new BaseQuality(metadata, t.getValue());
    }
    ).collect(Collectors.toList());
    mzQC = new MzQC(
            "nils.hoffmann@cebitec.uni-bielefeld.de",
            "Nils Hoffmann",
            Arrays.asList(
                    new ControlledVocabulary(
                            "Proteomics Standards Initiative Mass Spectrometry Ontology",
                            new URI("https://github.com/HUPO-PSI/psi-ms-CV/releases/download/v4.1.103/psi-ms.obo"),
                            "4.1.103"
                    )
            ),
            OffsetDateTime.now(),
            "MzQC for basic QC information on MetaboLights dataset MTBLS1375",
            bqs,
            Collections.emptyList(),
            "1.0.0");
    Set<ValidationMessage> messages = Converter.validate(mzQC);
    System.out.println("Validation messages: " + messages);
    if(!messages.isEmpty()) {
      System.err.println("Validation failed with "+messages.size()+" messages!");
    } else {
      System.out.println("Validation successful!");
      ObjectWriter writer = prepareJsonWriter();
      writer.writeValue(new File(file + ".mzQC"), new Coordinate(mzQC));
      // print the first 800 characters of the mzQC file
      System.out.println(writer.writeValueAsString(new Coordinate(mzQC)).substring(0, 799));
    }
} catch (URISyntaxException | IOException ex) {
    System.err.println("Exception: "+ex.getLocalizedMessage());
}

Validation messages: []
Validation successful!
{
  "mzQC" : {
    "contactAddress" : "nils.hoffmann@cebitec.uni-bielefeld.de",
    "contactName" : "Nils Hoffmann",
    "controlledVocabularies" : [ {
      "name" : "Proteomics Standards Initiative Mass Spectrometry Ontology",
      "uri" : "https://github.com/HUPO-PSI/psi-ms-CV/releases/download/v4.1.103/psi-ms.obo",
      "version" : "4.1.103"
    } ],
    "creationDate" : "2022-10-05T11:32:03.293753141Z",
    "description" : "MzQC for basic QC information on MetaboLights dataset MTBLS1375",
    "runQualities" : [ {
      "metadata" : {
        "analysisSoftware" : [ {
          "accession" : "MS:1000799",
          "name" : "custom unreleased software tool",
          "value" : "jmzqc",
          "uri" : "https://github.com/MS-Quality-hub/jmzqc",
          "version" : "1.0.0-RC1"
   


In [ ]:
import org.knowm.xchart.*;

if(mzQC!=null) {
  System.out.println("MzQC is not null!");
}

CategoryChart chart = new CategoryChartBuilder().width(800).height(600).title("m/z Range").build();

 